In [11]:
import pandas as pd
import numpy as np
import random
import datetime

In [12]:
def gerar_dados(tempo_seg):
    dados = {
        'Data': [],
        'Falha': [],
        'Status_Estacao': [],
        'Consertado': []
    }
    data = datetime.datetime(2022, 1, 16, 8, 0, 0) # Ano, mês, dia, hora, minuto, segundo
    status_maquina = 1  # Começa disponível
    Consertado = 0  # Começa sem pedido finalizado
    aux = 0
    aux_falha = 0
    for seg in range(1, tempo_seg + 1):
        
        Consertado = 0
        falha = 0
        
        #Limita a dia de semana
        if data.weekday() >= 5:
            tempoVoltar = 7 - data.weekday()
            data += datetime.timedelta(days=tempoVoltar)

        #Limita ao horario de 8 as 12 13 as 17
        if data.hour >= 12 and data.hour <= 13:
            data += datetime.timedelta(hours=1)
        if data.hour > 17:
            data += datetime.timedelta(hours=15)

        #Limitações logicas sobre entrar em falha e conserto
        if status_maquina == 1:
            aux_falha = random.randint(0,9)
            if aux_falha == 0:
                falha = 1
                status_maquina = 0
        elif status_maquina == 0:
            aux = random.randint(0,9)
            if aux == 0:
                Consertado = 1
                status_maquina = 1

        # Adiciona os dados para a linha atual no df
        dados['Data'].append(data)
        dados['Falha'].append(falha)
        dados['Status_Estacao'].append(status_maquina)
        dados['Consertado'].append(Consertado)
        data += datetime.timedelta(seconds=15)

    return pd.DataFrame(dados)

In [22]:
qtdGerada = 1000000  # Quantidade de dados a gerar por linha, intervalo entre linhas é 15 segundos.

# Estação de Saída

In [13]:
dados_saída = gerar_dados(qtdGerada)

In [14]:
df_saida = dados_saída.loc[:, ['Data', 'Status_Estacao']]

In [15]:
df_saida["data_abreviada"] = df_saida['Data'].dt.strftime('%Y-%m-%d')
df_saida["data_abreviada"] =  pd.to_datetime(df_saida["data_abreviada"])

In [16]:
df_saida['Disponibilidade'] = ''
anos = df_saida['data_abreviada'].dt.year.unique()
meses = df_saida['data_abreviada'].dt.month.unique()
dias = df_entrada['data_abreviada'].dt.day.unique()
for ano in anos: 
    for mes in meses:
        for dia in dias:
            df_aux = df_entrada[(df_saida['Data'].dt.year == ano) & (df_saida['Data'].dt.month == mes) & (df_saida['Data'].dt.day == dia)]
            try:
                df_saida.loc[(df_saida['Data'].dt.year == ano) & (df_saida['Data'].dt.month == mes) \
                            & (df_saida['Data'].dt.day == dia), "Disponibilidade"] = \
                                (df_aux.loc[df_aux['Status_Estacao'] == 1, 'Data'].max() - \
                                df_aux.loc[df_aux['Status_Estacao'] == 1,'Data'].min())/(df_aux.loc[:,'Data'].max() - \
                                                                                        df_aux.loc[:, 'Data'].min())
            except ZeroDivisionError:
                df_saida.loc[(df_saida['Data'].dt.year == ano) & (df_saida['Data'].dt.month == mes) \
                            & (df_saida['Data'].dt.day == dia), "Disponibilidade"] = 0

In [17]:
df_saida['Disponibilidade'] = df_saida['Disponibilidade'].fillna(0)

C:\Users\jhaid\AppData\Local\Temp\ipykernel_18604\3263956128.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_saida['Disponibilidade'] = df_saida['Disponibilidade'].fillna(0)


In [18]:
df_saida = df_saida.groupby(['data_abreviada']).agg({'Disponibilidade': 'mean'})

In [19]:
df_saida = df_saida.reset_index()

In [1]:
df_saida

NameError: name 'df_saida' is not defined

In [21]:
df_saida.to_csv('dados_saida.csv', index=False)

# Estação de Entrada

In [23]:
dados_entrada = gerar_dados(qtdGerada)
df_entrada = dados_entrada.loc[:, ['Data', 'Status_Estacao']]
df_entrada["data_abreviada"] = df_entrada['Data'].dt.strftime('%Y-%m-%d')
df_entrada["data_abreviada"] =  pd.to_datetime(df_entrada["data_abreviada"])


df_entrada['Disponibilidade'] = ''
anos = df_entrada['data_abreviada'].dt.year.unique()
meses = df_entrada['data_abreviada'].dt.month.unique()
dias = df_entrada['data_abreviada'].dt.day.unique()
for ano in anos: 
    for mes in meses:
        for dia in dias:
            df_aux = df_entrada[(df_entrada['Data'].dt.year == ano) & (df_entrada['Data'].dt.month == mes) & (df_entrada['Data'].dt.day == dia)]
            try:
                df_entrada.loc[(df_entrada['Data'].dt.year == ano) & (df_entrada['Data'].dt.month == mes) \
                            & (df_entrada['Data'].dt.day == dia), "Disponibilidade"] = \
                                (df_aux.loc[df_aux['Status_Estacao'] == 1, 'Data'].max() - \
                                df_aux.loc[df_aux['Status_Estacao'] == 1,'Data'].min())/(df_aux.loc[:,'Data'].max() - \
                                                                                        df_aux.loc[:, 'Data'].min())
            except ZeroDivisionError:
                df_entrada.loc[(df_entrada['Data'].dt.year == ano) & (df_entrada['Data'].dt.month == mes) \
                            & (df_entrada['Data'].dt.day == dia), "Disponibilidade"] = 0

In [24]:
df_entrada['Disponibilidade'] = df_entrada['Disponibilidade'].fillna(0)
df_entrada = df_entrada.groupby(['data_abreviada']).agg({'Disponibilidade': 'mean'})
df_entrada = df_entrada.reset_index()
df_entrada

C:\Users\jhaid\AppData\Local\Temp\ipykernel_18604\3942035672.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_entrada['Disponibilidade'] = df_entrada['Disponibilidade'].fillna(0)


,data_abreviada,Disponibilidade
0,2022-01-17,0.997916
1,2022-01-18,0.986105
2,2022-01-19,0.995368
3,2022-01-20,0.989810
4,2022-01-21,0.993979
...,...,...
710,2024-04-24,0.995368
711,2024-04-25,0.993979
712,2024-04-26,1.000000
713,2024-04-27,0.000000


In [25]:
df_entrada.to_csv('dados_entrada.csv', index=False)

# Estação de Transporte

In [26]:
dados_transporte = gerar_dados(qtdGerada)
df_transporte = dados_transporte.loc[:, ['Data', 'Status_Estacao']]
df_transporte["data_abreviada"] = df_transporte['Data'].dt.strftime('%Y-%m-%d')
df_transporte["data_abreviada"] =  pd.to_datetime(df_transporte["data_abreviada"])


df_transporte['Disponibilidade'] = ''
anos = df_transporte['data_abreviada'].dt.year.unique()
meses = df_transporte['data_abreviada'].dt.month.unique()
dias = df_transporte['data_abreviada'].dt.day.unique()
for ano in anos: 
    for mes in meses:
        for dia in dias:
            df_aux = df_transporte[(df_transporte['Data'].dt.year == ano) & (df_transporte['Data'].dt.month == mes) & (df_transporte['Data'].dt.day == dia)]
            try:
                df_transporte.loc[(df_transporte['Data'].dt.year == ano) & (df_transporte['Data'].dt.month == mes) \
                            & (df_transporte['Data'].dt.day == dia), "Disponibilidade"] = \
                                (df_aux.loc[df_aux['Status_Estacao'] == 1, 'Data'].max() - \
                                df_aux.loc[df_aux['Status_Estacao'] == 1,'Data'].min())/(df_aux.loc[:,'Data'].max() - \
                                                                                        df_aux.loc[:, 'Data'].min())
            except ZeroDivisionError:
                df_transporte.loc[(df_transporte['Data'].dt.year == ano) & (df_transporte['Data'].dt.month == mes) \
                            & (df_transporte['Data'].dt.day == dia), "Disponibilidade"] = 0

In [27]:
df_transporte['Disponibilidade'] = df_transporte['Disponibilidade'].fillna(0)
df_transporte = df_transporte.groupby(['data_abreviada']).agg({'Disponibilidade': 'mean'})
df_transporte = df_transporte.reset_index()

df_transporte.to_csv('dados_transporte.csv', index=False)

C:\Users\jhaid\AppData\Local\Temp\ipykernel_18604\816845517.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_transporte['Disponibilidade'] = df_transporte['Disponibilidade'].fillna(0)


# Estação de Inspeção

In [28]:
dados_inspecao = gerar_dados(qtdGerada)
df_inspecao = dados_inspecao.loc[:, ['Data', 'Status_Estacao']]
df_inspecao["data_abreviada"] = df_inspecao['Data'].dt.strftime('%Y-%m-%d')
df_inspecao["data_abreviada"] =  pd.to_datetime(df_inspecao["data_abreviada"])


df_inspecao['Disponibilidade'] = ''
anos = df_inspecao['data_abreviada'].dt.year.unique()
meses = df_inspecao['data_abreviada'].dt.month.unique()
dias = df_inspecao['data_abreviada'].dt.day.unique()
for ano in anos: 
    for mes in meses:
        for dia in dias:
            df_aux = df_inspecao[(df_inspecao['Data'].dt.year == ano) & (df_inspecao['Data'].dt.month == mes) & (df_inspecao['Data'].dt.day == dia)]
            try:
                df_inspecao.loc[(df_inspecao['Data'].dt.year == ano) & (df_inspecao['Data'].dt.month == mes) \
                            & (df_inspecao['Data'].dt.day == dia), "Disponibilidade"] = \
                                (df_aux.loc[df_aux['Status_Estacao'] == 1, 'Data'].max() - \
                                df_aux.loc[df_aux['Status_Estacao'] == 1,'Data'].min())/(df_aux.loc[:,'Data'].max() - \
                                                                                        df_aux.loc[:, 'Data'].min())
            except ZeroDivisionError:
                df_inspecao.loc[(df_inspecao['Data'].dt.year == ano) & (df_inspecao['Data'].dt.month == mes) \
                            & (df_inspecao['Data'].dt.day == dia), "Disponibilidade"] = 0

In [29]:
df_inspecao['Disponibilidade'] = df_inspecao['Disponibilidade'].fillna(0)
df_inspecao = df_inspecao.groupby(['data_abreviada']).agg({'Disponibilidade': 'mean'})
df_inspecao = df_inspecao.reset_index()

df_inspecao.to_csv('dados_inspecao.csv', index=False)

C:\Users\jhaid\AppData\Local\Temp\ipykernel_18604\4031482446.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_inspecao['Disponibilidade'] = df_inspecao['Disponibilidade'].fillna(0)


# Estação de Limpeza

In [30]:
dados_limpeza = gerar_dados(qtdGerada)
df_limpeza = dados_limpeza.loc[:, ['Data', 'Status_Estacao']]
df_limpeza["data_abreviada"] = df_limpeza['Data'].dt.strftime('%Y-%m-%d')
df_limpeza["data_abreviada"] =  pd.to_datetime(df_limpeza["data_abreviada"])


df_limpeza['Disponibilidade'] = ''
anos = df_limpeza['data_abreviada'].dt.year.unique()
meses = df_limpeza['data_abreviada'].dt.month.unique()
dias = df_limpeza['data_abreviada'].dt.day.unique()
for ano in anos: 
    for mes in meses:
        for dia in dias:
            df_aux = df_limpeza[(df_limpeza['Data'].dt.year == ano) & (df_limpeza['Data'].dt.month == mes) & (df_limpeza['Data'].dt.day == dia)]
            try:
                df_limpeza.loc[(df_limpeza['Data'].dt.year == ano) & (df_limpeza['Data'].dt.month == mes) \
                            & (df_limpeza['Data'].dt.day == dia), "Disponibilidade"] = \
                                (df_aux.loc[df_aux['Status_Estacao'] == 1, 'Data'].max() - \
                                df_aux.loc[df_aux['Status_Estacao'] == 1,'Data'].min())/(df_aux.loc[:,'Data'].max() - \
                                                                                        df_aux.loc[:, 'Data'].min())
            except ZeroDivisionError:
                df_limpeza.loc[(df_limpeza['Data'].dt.year == ano) & (df_limpeza['Data'].dt.month == mes) \
                            & (df_limpeza['Data'].dt.day == dia), "Disponibilidade"] = 0

In [31]:
df_limpeza['Disponibilidade'] = df_limpeza['Disponibilidade'].fillna(0)
df_limpeza = df_limpeza.groupby(['data_abreviada']).agg({'Disponibilidade': 'mean'})
df_limpeza = df_limpeza.reset_index()

df_limpeza.to_csv('dados_limpeza.csv', index=False)

C:\Users\jhaid\AppData\Local\Temp\ipykernel_18604\4173822122.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_limpeza['Disponibilidade'] = df_limpeza['Disponibilidade'].fillna(0)
